

For this analysis, we're modeling a **4:1 split-rate tax** where land is taxed at four times the rate of buildings.

## Policy Definition for South Bend

For this analysis, we're modeling:
- **Revenue-neutral** property tax split for the South Bend School Corporation
- **4:1 land-to-building tax ratio** (partial LVT shift)
- **Existing exemptions and abatements** continue to apply
- **Focus on school corporation taxes** (which bypass Indiana's property tax caps)

Let's begin by importing the necessary libraries and utility functions.


In [10]:
import sys
import pandas as pd
sys.path.append('..')  # Add parent directory to path
from cloud_utils import get_feature_data, get_feature_data_with_geometry
from lvt_utils import model_split_rate_tax, calculate_current_tax
from census_utils import get_census_data, get_census_blockgroups_shapefile, get_census_data_with_boundaries, match_to_census_blockgroups



## Step 1: Getting the Data

The first step in modeling an LVT shift is obtaining property tax data. Most counties make this information publicly accessible through open data portals or GIS systems.

For St. Joseph County (which includes South Bend), we can access parcel data through their ArcGIS services. The base URL below provides access to various property datasets including:

- **Parcel_Civic**: Main parcel dataset with tax information, property types, and assessed values
- **parcel_boundaries**: Geographic boundaries for spatial analysis

### Key Data Elements We Need:
- **Full Market Value (FMV)**: Total assessed property value
- **Land Value**: Assessed value of land only  
- **Improvement Value**: Assessed value of buildings/structures
- **Exemption amounts**: Various tax exemptions applied
- **Property characteristics**: Type, location, tax district

Let's fetch the main parcel dataset:


In [ ]:

# Base URL for the ArcGIS services
base_url = "https://services.arcgis.com/OjftlhRHkAABcyiF/arcgis/rest/services"
# Fetch the main parcel dataset with tax info
parcel_civic_df = get_feature_data('Parcel_Civic', base_url)


## Step 2: Filtering to South Bend School Corporation

Now we need to filter our dataset to include only properties within our area of interest. For this analysis, we're focusing on the South Bend School Corporation tax district.

### Why Focus on School Corporation Taxes?

In Indiana, most properties hit statutory tax caps that limit property tax increases. However, the South Bend School Corporation passed a referendum that allows them to exceed these caps, making it an ideal case study for LVT modeling where tax changes can actually take effect.

We'll filter for:
- Properties in South Bend city limits (`PROP_CITY` contains 'SOUTH BEND')  
- Properties in South Bend tax districts (`TAXDIST` contains 'SB')


In [ ]:
df = parcel_civic_df.copy()[parcel_civic_df['PROP_CITY'].str.upper().str.contains('SOUTH BEND', na=False)]
df = df[df['TAXDIST'].str.contains('SB')].copy()

## Step 3: Recreating Current Property Tax Revenue

Before we can model an LVT shift, we need to accurately recreate the current property tax system. This validation step ensures our dataset correctly reflects the real-world tax landscape.

### Key Components:
- **Millage Rate**: $3.3 per $1,000 in assessed value (from South Bend School Corporation budget)
- **Exemptions**: Various exemptions that reduce taxable value
- **Exempt Properties**: Fully exempt properties (marked in `PROPTYPE`)

### The Process:
1. Calculate total exemptions from all exemption amount fields
2. Identify fully exempt properties  
3. Calculate taxable value (land + improvements - exemptions)
4. Apply millage rate to get current tax liability
5. Verify total revenue matches published budget expectations (~$27 million)

This step is crucial - if we can't accurately recreate current taxes, our LVT projections won't be reliable.


In [ ]:

millage_rate = 3.3
df['millage_rate'] = millage_rate

# 1. Calculate current tax
df['exemption_flag'] = df['PROPTYPE'].apply(lambda x: 1 if 'Exempt' in x else 0)
print(f"Number of exempt properties: {df['exemption_flag'].sum():,} ({df['exemption_flag'].mean()*100:.1f}%)")

df['total_exemptions'] = df[['ExemptAmt1', 'ExemptAmt2', 'ExemptAmt3', 'ExemptAmt4', 'ExemptAmt5', 'ExemptAmt6']].sum(axis=1)
# Calculate taxable value (land + improvements)
df['taxable_value'] = df['REALLANDVA'] + df['REALIMPROV']

current_revenue, second_revenue,df = calculate_current_tax(
    df=df, 
    tax_value_col='taxable_value',
    millage_rate_col='millage_rate',
    exemption_col='total_exemptions',
    exemption_flag_col='exemption_flag'
)


print(f"Total number of properties: {len(df):,}")
print(f"Current annual revenue with ${millage_rate*1000}/1000 millage rate: ${current_revenue:,.2f}")
print(f"Total land value: ${df['REALLANDVA'].sum():,.2f}")
print(f"Total improvement value: ${df['REALIMPROV'].sum():,.2f}")



## Step 4: Modeling the Split-Rate Land Value Tax

Now for the exciting part - modeling the LVT shift! We'll create a revenue-neutral policy that taxes land at 4 times the rate of buildings.

### The Split-Rate Formula

Under our proposed system:
- **Buildings** are taxed at a lower rate (Building Millage)  
- **Land** is taxed at 4x that rate (4 × Building Millage)
- **Total revenue** remains the same as current system

The formula to solve for the building millage rate is:
```
Current Revenue = (Building Millage × Total Taxable Buildings) + (4 × Building Millage × Total Taxable Land)
```

### Handling Exemptions in Split-Rate System

Since we want to maintain existing exemptions, we need to:
1. Apply exemptions to building value first
2. If exemptions exceed building value, apply remainder to land value
3. Calculate separate taxable values for land and buildings

This ensures properties don't over-benefit from exemptions and maintains the intent of existing tax policy.


In [ ]:
# Calculate split-rate tax using model_split_rate_tax function
land_millage, improvement_millage, new_revenue, df = model_split_rate_tax(
    df=df,
    land_value_col='REALLANDVA',
    improvement_value_col='REALIMPROV',
    current_revenue=current_revenue,
    land_improvement_ratio=4  # 4:1 ratio as specified
)

# Calculate tax changes manually since they're not being added by the function
df['NEW_TAX'] = (df['REALLANDVA'] * land_millage/1000) + (df['REALIMPROV'] * improvement_millage/1000)
df['TAX_CHANGE'] = df['new_tax'] - df['current_tax']
df['TAX_CHANGE_PCT'] = (df['TAX_CHANGE'] / df['current_tax']) * 100


## Step 5: Understanding Property Types and Impacts

With our split-rate tax calculated, we can now analyze which property types are most affected. Understanding the distribution of tax impacts across different property categories is crucial for policy makers and stakeholders.

### Property Type Analysis

We'll examine how the tax burden shifts across:
- **Residential properties** (single-family, multi-family, condos)
- **Commercial properties** (retail, office, industrial)  
- **Vacant land** (often sees largest increases under LVT)
- **Exempt properties** (government, religious, charitable)

### Key Metrics to Track:
- **Count**: Number of properties in each category
- **Median tax change**: Typical impact (less affected by outliers)
- **Average percentage change**: Overall magnitude of impact
- **Percentage with increases**: How many properties see tax increases

This analysis helps identify which sectors benefit from the LVT shift (typically developed properties) and which see increased burden (typically land-intensive properties with low improvement ratios).


In [ ]:
# For each column, show top 10 most common values and their counts
columns_to_analyze = ['CLASSCODE', 'TOWNSHIP', 'TAXDIST', 'Neighborho', 'PROPTYPE', 
                     'TAXTYPE', 'TIFAREAUID', 'LEGALDESCR']

for col in columns_to_analyze:
    print(f"\nTop 10 values for {col}:")
    value_counts = df[col].value_counts().head(10)
    print(value_counts)
    print(f"Total unique values: {df[col].nunique()}")
    print("-" * 50)

# Let's also look at some basic statistics about these groups
print("\nMedian tax changes by various groupings:")

for col in ['CLASSCODE', 'TOWNSHIP', 'TAXDIST', 'PROPTYPE']:
    print(f"\nMedian tax change by {col}:")
    median_changes = df.groupby(col)['TAX_CHANGE'].agg([
        'count',
        'median',
        lambda x: (x > 0).mean() * 100  # Percentage with increase
    ]).round(2)
    median_changes.columns = ['Count', 'Median Change ($)', '% With Increase']
    print(median_changes.sort_values('Count', ascending=False).head(10))

In [16]:
def categorize_property_type(prop_type):
    # Detailed single-family home categorization
    if "1 Family Dwell - Platted Lot" in prop_type:
        return "Single Family"
    elif "1 Family Dwell - Unplatted (0 to 9.99 Acres)" in prop_type:
        return "Single Family - Unplatted Small Acreage"
    elif "1 Family Dwell - Unplatted (10 to 19.99 Acres)" in prop_type:
        return "Single Family - Unplatted Medium Acreage"
    elif "1 Family Dwell - Unplatted" in prop_type:
        return "Single Family - Unplatted Large Acreage"

    # Rest of your existing categorization
    category_mapping = {
        "Small Multi-Family (2-19 units)": ["2 Family", "3 Family", "4 to 19 Family"],
        "Large Multi-Family (20+ units)": ["20 to 39 Family", "40 or More Family"],
        "Condominiums": ["Condominium"],
        "Mobile/Manufactured Homes": ["Mobile", "Manufactured"],
        "Retail Commercial": ["Retail", "Shop", "Store", "Market", "Department", "Shopping Center"],
        "Office Commercial": ["Office", "Medical", "Bank", "Saving"],
        "Food/Hospitality": ["Restaurant", "Bar", "Hotel", "Motel", "Food"],
        "Industrial": ["Industrial", "Manufacturing", "Warehouse", "Assembly", "Factory"],
        "Vacant Land": ["Vacant"],
        "Parking": ["Parking"],
        "Government": ["Exempt Municipality", "Exempt County", "Exempt State", "Exempt United States",
                      "Exempt Board of Education", "Exempt Township"],
        "Religious": ["Exempt Religious", "Exempt Church", "Exempt Chapel", "Exempt Mosque",
                     "Exempt Synagogue", "Exempt Temple"],
        "Charitable": ["Exempt Charitable"],
        "Agricultural": ["Farm", "Agricultural", "Grain", "Livestock", "Dairy", "Nursery", "Poultry"]
    }

    # Check for matches
    for category, keywords in category_mapping.items():
        if any(keyword in prop_type for keyword in keywords):
            return category

    # Handle remaining cases
    if "Exempt" in prop_type:
        return "Other Exempt"
    elif "Commercial" in prop_type:
        return "Other Commercial"
    elif "Residential" in prop_type:
        return "Other Residential"
    else:
        return "Other"

# Apply the function to the DataFrame
df['PROPERTY_CATEGORY'] = df['PROPTYPE'].apply(categorize_property_type)

### Creating Detailed Property Categories

To better understand impacts, we'll create a detailed property categorization system that groups similar property types together. This makes the analysis more meaningful and interpretable.

The function below categorizes properties into groups like:
- **Single Family** (with subcategories by lot size)
- **Multi-Family** (small vs. large)
- **Commercial** (by type: retail, office, industrial)
- **Exempt** (by type: government, religious, charitable)

This categorization helps us understand not just that "residential" properties are affected, but specifically which types of residential properties see the biggest changes.


In [ ]:
# Create a summary DataFrame grouped by PROPTYPE
proptype_analysis = df.groupby('PROPERTY_CATEGORY').agg({
    'TAX_CHANGE_PCT': 'mean',  # Average percentage change
    'TAX_CHANGE': 'median',    # Median dollar change
    'PARCELID': 'count'        # Count of properties
}).round(2)

# Add percentage that increase
proptype_increases = df.groupby('PROPERTY_CATEGORY').agg({
    'TAX_CHANGE': lambda x: (x > 0).mean() * 100  # Percentage with increase
}).round(2)

proptype_analysis['Percent_Increased'] = proptype_increases['TAX_CHANGE']

# Rename columns for clarity
proptype_analysis.columns = [
    'Avg_Pct_Change',
    'Median_Dollar_Change',
    'Property_Count',
    'Pct_Properties_Increased'
]

# Sort by count of properties (descending)
proptype_analysis = proptype_analysis.sort_values('Property_Count', ascending=False)

# Print results
print("Analysis by Property Type:\n")
print("Note: All monetary values in dollars, percentages shown as %\n")
print(proptype_analysis.to_string())

# Print some summary statistics
print("\nOverall Summary:")
print(f"Total properties analyzed: {proptype_analysis['Property_Count'].sum():,}")
print(f"Overall median dollar change: ${df['TAX_CHANGE'].median():,.2f}")
print(f"Overall average percent change: {df['TAX_CHANGE_PCT'].mean():.2f}%")
print(f"Overall percent of properties with increase: {(df['TAX_CHANGE'] > 0).mean()*100:.2f}%")

### Summary of Tax Impacts by Property Category

Now we can see the clear patterns of how different property types are affected by the LVT shift. This table will show us:

- **Which property types benefit** (negative changes = tax decreases)
- **Which property types pay more** (positive changes = tax increases)  
- **How concentrated the impacts are** (median vs. average differences)
- **What percentage of each type sees increases**

Generally, we expect:
- **Developed properties** (houses, commercial buildings) to see tax **decreases**
- **Vacant land** to see the **largest increases** 
- **Properties with high improvement-to-land ratios** to benefit most


In [ ]:
boundary_gdf = get_feature_data_with_geometry('parcel_boundaries', base_url=base_url)


## Step 6: Adding Geographic Context

To make our analysis spatially-aware, we need to add geographic boundaries to our parcel data. This enables us to:

- **Create maps** showing tax changes across the city
- **Analyze patterns by neighborhood** or district  
- **Combine with demographic data** for equity analysis
- **Present results visually** to stakeholders

We'll fetch the parcel boundary data from the same ArcGIS service that contains the geometric information for each property.


In [ ]:

print(len(df))

# Merge with our tax analysis data
merged_gdf = df.merge(
    df,
    on='PARCELID',
    how='inner'
)

print(f"\nMerged data has {len(merged_gdf)} parcels")

### Merging Tax Analysis with Geographic Data

Here we combine our tax analysis results with the geographic boundaries. This creates a spatially-enabled dataset that allows us to:

1. **Map tax changes** across South Bend
2. **Identify spatial patterns** in tax impacts
3. **Prepare for demographic analysis** by having geographic context

The merge should give us the same number of records as our original analysis, now with geographic coordinates for each parcel.


In [ ]:
# Get census data for St. Joseph County (FIPS code: 18141)
census_data, census_boundaries = get_census_data_with_boundaries(
    fips_code='18141',  # Indiana (18) + St. Joseph County (141)
    year=2022,
    api_key='YOUR_API_KEY'  # Replace with your actual Census API key
)

# Set CRS for census boundaries before merging
census_boundaries = census_boundaries.set_crs(epsg=4326)  # Assuming WGS84 coordinate system
boundary_gdf = boundary_gdf.set_crs(epsg=4326)  # Set same CRS for boundary data

# Merge census data with our parcel boundaries
merged_gdf = match_to_census_blockgroups(
    gdf=boundary_gdf,
    census_gdf=census_boundaries,
    join_type="left"
)

# Merge the census data back onto the original dataframe
df = df.merge(
    merged_gdf,
    left_on='PARCELID',
    right_on='PARCELID',
    how='left'
)

print(f"Number of census blocks: {len(census_boundaries)}")
print(f"Number of census data: {len(census_data)}")
print(f"Number of parcels with census data: {len(df)}")

## Step 7: Demographic and Equity Analysis

One of the most important aspects of LVT analysis is understanding the **equity implications** - how does the tax shift affect different income levels and demographic groups?

### Adding Census Data

We'll match each property to its Census Block Group and pull demographic data including:
- **Median household income** 
- **Racial/ethnic composition**
- **Population characteristics**

### Why This Matters

Policy makers need to understand:
- Does the LVT shift disproportionately burden low-income neighborhoods?
- Are there racial equity implications?  
- Does the policy align with broader equity goals?

**Note**: You'll need a Census API key for this section. Get one free at: https://api.census.gov/data/key_signup.html


In [ ]:
print("DataFrame columns:")
print(df.columns.tolist())


### Exploring the Enhanced Dataset

With census data merged in, our dataset now contains both property tax information and demographic context. Let's explore what variables we now have available for analysis.

This enhanced dataset allows us to examine relationships between:
- Property characteristics and demographics
- Tax impacts and neighborhood income levels
- Geographic patterns in tax burden shifts


In [ ]:
# Display all columns with maximum width
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
display(df.head())


### Viewing the Complete Dataset

Let's examine our enhanced dataset with all the variables we've created and merged. This gives us a comprehensive view of each property with:

- **Property characteristics** (type, value, location)
- **Current tax calculations** 
- **New LVT calculations**
- **Tax change impacts**
- **Demographic context** (income, race/ethnicity)

This rich dataset forms the foundation for sophisticated equity and impact analysis.


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

def filter_data(df):
    """Filter data to remove negative median incomes and create non-vacant subset"""
    df_filtered = df[df['median_income'] > 0].copy()
    non_vacant_df = df[df['PROPERTY_CATEGORY'] != 'Vacant Land'].copy()
    return df_filtered, non_vacant_df

def calculate_block_group_summary(df):
    """Calculate summary statistics for census block groups"""
    summary = df.groupby('std_geoid').agg(
        median_income=('median_income', 'first'),
        minority_pct=('minority_pct', 'first'),
        black_pct=('black_pct', 'first'),
        total_current_tax=('current_tax', 'sum'),
        total_new_tax=('new_tax', 'sum'),
        mean_tax_change=('TAX_CHANGE', 'mean'),
        median_tax_change=('TAX_CHANGE', 'median'),
        median_tax_change_pct=('TAX_CHANGE_PCT', 'median'),
        parcel_count=('TAX_CHANGE', 'count'),
        has_vacant_land=('PROPERTY_CATEGORY', lambda x: 'Vacant Land' in x.values)
    ).reset_index()
    
    summary['mean_tax_change_pct'] = ((summary['total_new_tax'] - summary['total_current_tax']) / 
                                    summary['total_current_tax']) * 100
    return summary

def create_scatter_plot(data, x_col, y_col, ax, title, xlabel, ylabel):
    """Create a scatter plot with trend line"""
    sns.scatterplot(
        data=data,
        x=x_col,
        y=y_col,
        size='parcel_count',
        sizes=(20, 200),
        alpha=0.7,
        ax=ax
    )
    
    ax.axhline(y=0, color='r', linestyle='--')
    
    x = data[x_col].dropna()
    y = data[y_col].dropna()
    mask = ~np.isnan(x) & ~np.isnan(y)
    
    if len(x[mask]) > 1:
        z = np.polyfit(x[mask], y[mask], 1)
        p = np.poly1d(z)
        ax.plot(x[mask], p(x[mask]), "r--")
    
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    ax.set_title(title)

def plot_comparison(data1, data2, x_col, y_col, title_prefix, xlabel):
    """Create side-by-side comparison plots"""
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 8))
    
    create_scatter_plot(data1, x_col, y_col, ax1, 
                       f'{title_prefix} - All Properties', xlabel, 'Mean Tax Change (%)')
    create_scatter_plot(data2, x_col, y_col, ax2,
                       f'{title_prefix} - Excluding Vacant Land', xlabel, 'Mean Tax Change (%)')
    
    plt.tight_layout()
    plt.show()

def calculate_correlations(data1, data2):
    """Calculate correlations between variables"""
    correlations = {}
    for df, suffix in [(data1, 'all'), (data2, 'non_vacant')]:
        correlations[f'income_mean_{suffix}'] = df[['median_income', 'mean_tax_change_pct']].corr().iloc[0, 1]
        correlations[f'income_median_{suffix}'] = df[['median_income', 'median_tax_change_pct']].corr().iloc[0, 1]
        correlations[f'minority_mean_{suffix}'] = df[['minority_pct', 'mean_tax_change_pct']].corr().iloc[0, 1]
        correlations[f'black_mean_{suffix}'] = df[['black_pct', 'mean_tax_change_pct']].corr().iloc[0, 1]
    return correlations

def create_quintile_summary(df, group_col, value_col):
    """Create summary statistics by quintiles"""
    df[f'{group_col}_quintile'] = pd.qcut(df[group_col], 5, 
                                         labels=["Q1 (Lowest)", "Q2", "Q3", "Q4", "Q5 (Highest)"])
    
    summary = df.groupby(f'{group_col}_quintile').agg(
        count=('TAX_CHANGE', 'count'),
        mean_tax_change=('TAX_CHANGE', 'mean'),
        median_tax_change=('TAX_CHANGE', 'median'),
        mean_value=(value_col, 'mean')
    ).reset_index()
    
    return summary

# Main execution
gdf_filtered, non_vacant_gdf = filter_data(df)
print(f"Number of rows in gdf_filtered: {len(gdf_filtered)}")
print(f"Number of rows in non_vacant_gdf: {len(non_vacant_gdf)}")

# Calculate block group summaries
census_block_groups = calculate_block_group_summary(gdf_filtered)
non_vacant_block_summary = calculate_block_group_summary(non_vacant_gdf)

# Create comparison plots
plot_comparison(census_block_groups, non_vacant_block_summary, 
               'median_income', 'mean_tax_change_pct', 
               'Mean Tax Change vs. Median Income', 
               'Median Income by Census Block Group ($)')

plot_comparison(census_block_groups, non_vacant_block_summary,
               'minority_pct', 'mean_tax_change_pct',
               'Mean Tax Change vs. Minority Percentage',
               'Minority Population Percentage by Census Block Group')

plot_comparison(census_block_groups, non_vacant_block_summary,
               'black_pct', 'mean_tax_change_pct',
               'Mean Tax Change vs. Black Percentage',
               'Black Population Percentage by Census Block Group')

# Calculate and print correlations
correlations = calculate_correlations(census_block_groups, non_vacant_block_summary)
for key, value in correlations.items():
    print(f"Correlation {key}: {value:.4f}")

# Create and display quintile summaries
income_quintile_summary = create_quintile_summary(gdf_filtered, 'median_income', 'median_income')
non_vacant_income_quintile_summary = create_quintile_summary(non_vacant_gdf, 'median_income', 'median_income')
minority_quintile_summary = create_quintile_summary(gdf_filtered, 'minority_pct', 'minority_pct')
non_vacant_minority_quintile_summary = create_quintile_summary(non_vacant_gdf, 'minority_pct', 'minority_pct')

print("\nTax impact by income quintile (all properties):")
display(income_quintile_summary)
print("\nTax impact by income quintile (excluding vacant land):")
display(non_vacant_income_quintile_summary)
print("\nTax impact by minority percentage quintile (all properties):")
display(minority_quintile_summary)
print("\nTax impact by minority percentage quintile (excluding vacant land):")
display(non_vacant_minority_quintile_summary)
